# Modelo de Estimación del Precio de Viviendas
## Step 1: Business Understanding
Este dataset contiene información recolectada de precios y características de 142 mil viviendas en Colombia. La información se encuentra disponible públicamente en el repositorio Kaggle: https://www.kaggle.com/datasets/danieleduardofajardo/colombia-house-prediction

### Cargamos las librerías

In [ ]:
from teradataml import create_context, DataFrame, get_context, copy_to_sql, in_schema, remove_context
from teradataml.dataframe.sql_functions import case
import tdconnect
import pandas as pd
import numpy as np
import getpass as gp

from teradataml import *
from teradataml.analytics.valib import *
configure.val_install_location = "TRNG_XSP"

### Creamos la conexión con Vantage

In [ ]:
# Establish connection to Teradata Vantage server (uses the Teradata SQL Driver for Python). 
td_context = create_context(host="tdprd.td.teradata.com", username="lc250058", password=gp.getpass(prompt='Password:'), logmech="LDAP")

### Creamos el DataFrame y traemos la cabecera

In [ ]:
tdPrecios = DataFrame(in_schema("ADLSLSAMER_MS_AZ", "Precio_Casas_Col"))
tdPrecios.head(10).to_pandas()

### Tamaño de la tabla

In [ ]:
tdPrecios.shape

## Step 2: Data Understanding
### Exploración de Valores

In [ ]:
explor = valib.Values(data=tdPrecios, columns="all")
explor.result.to_pandas()

### Explorando los Valores Atípicos (Outliers)

In [ ]:
out = valib.Statistics(data=tdPrecios, columns=["area", "valor"], extended_options="quantiles")
out.result.to_pandas()

### Distribución de Frecuencias por la Antiguedad

In [ ]:
tdAntig = valib.Frequency(data=tdPrecios, columns="antiguedad_original")
tdAntig.result.to_pandas().sort_values(['xval'])

## Step 3: Data Preparation
### Transformacion de Variables e Imputacion

In [ ]:
##dc = OneHotEncoder(style="dummy", values=["Entre 0 y 5 años", "Entre 5 y 10 años", "Entre 10 y 20 años","Más de 20 años"],columns="antiguedad_original", out_columns="antig")

In [ ]:
ndf = DataFrame.from_query("select id, area, habitaciones, CASE WHEN antiguedad_original='Entre 10 y 20 años' THEN 1 ELSE 0 END AS ant10_20, CASE WHEN antiguedad_original='Entre 0 y 5 años' THEN 1 ELSE 0 END AS ant0_5, CASE WHEN antiguedad_original='Entre 5 y 10 años' THEN 1 ELSE 0 END AS ant5_10, CASE WHEN antiguedad_original='Más de 20 años' THEN 1 ELSE 0 END AS ant20_mas, CASE WHEN antiguedad_original='1 a 8 años' THEN 1 ELSE 0 END AS ant1_8, CASE WHEN antiguedad_original='16 a 30 años' THEN 1 ELSE 0 END AS ant16_30, CASE WHEN antiguedad_original='9 a 15 años' THEN 1 ELSE 0 END AS ant9_15, CASE WHEN antiguedad_original='Más de 30 años' THEN 1 ELSE 0 END AS ant30_mas, CASE WHEN antiguedad_original='Menos de 1 año' THEN 1 ELSE 0 END AS ant1_menos, banos, garajes, estrato, valor, SAMPLEID as sid FROM ADLSLSAMER_MS_AZ.Precio_Casas_Col WHERE area between 20 and 2000 and valor between 50000000 and 5000000000 SAMPLE RANDOMIZED ALLOCATION 0.7, 0.3", True, "id")

In [ ]:
fn_1 = FillNa(style="literal", value=0, columns="garajes")
fn_2 = FillNa(style="literal", value=0, columns="estrato")
fn_3 = FillNa(style="literal", value=0, columns="banos")

In [ ]:
derive = Derive(formula="sqrt(x)", columns="valor", out_column="rvalor")

In [ ]:
retain = Retain(columns=["habitaciones", "area","ant0_5","ant5_10","ant10_20","ant20_mas","ant1_menos","ant1_8","ant9_15","ant16_30","ant30_mas","sid"])

In [ ]:
matriz = valib.Transform(data=ndf, fillna=[fn_1, fn_2, fn_3], derive=derive, retain=retain, key_columns="id", index_columns="id")
matriz.result.head().to_pandas()

### Matriz de Correlaciones para las variables numéricas

In [ ]:
Cor_Mat = valib.Matrix(data=matriz.result, columns='all', type="COR")
# Print the results.
Cor_Mat.result.to_pandas().sort_values(['rownum'])

### División de Muestras Train y Evaluation

In [ ]:
tbl_train=matriz.result[matriz.result["sid"]==1]
tbl_test=matriz.result[matriz.result["sid"]==2]

## Step 4: Modeling
### Generación del Modelo de Estimación de Precios

In [ ]:
tdModel = valib.LinReg(data=tbl_train,
                       columns="all",
                       exclude_columns=["id", "sid"],
                       stepwise=True,
                       response_column="rvalor")
tdModel.model.to_pandas()

## Step 5: Evaluation

In [ ]:
tdModel.statistical_measures.to_pandas()

In [ ]:
valib.LinRegEvaluator(data=tbl_train, model=tdModel.model)

In [ ]:
valib.LinRegEvaluator(data=tbl_test, model=tdModel.model)

### Step 6: Deployment

In [ ]:
tdScore = valib.LinRegPredict(data=tbl_test, model=tdModel.model, response_column="rvalue_estim")
tdScore.result.head().to_pandas()

In [ ]:
derive = Derive(formula="x*x", columns="rvalue_estim", out_column="valor_estim")
ScoreFinal = valib.Transform(data=tdScore.result, derive=derive, key_columns="id", index_columns="id")

In [ ]:
ScoreFinal.result.to_sql(schema_name="ADLSLSAMER_MS_AZ", table_name="Precio_Score")

Copyright 2022. Elaborado por Luis Cajachahua bajo licencia MIT